In [38]:
import numpy as np
import random
import math

In [39]:
# e_i,j = 0.1 for all b_i > b_j (beat the mean example 1)
K = 5 # number of bandits
T = 10^10 # time horizon

Eps = np.zeros((K, K))
for i in range(K):
    for j in range(i+1, K):
        Eps[i, j] = -0.1
        Eps[j, i] = 0.1

In [40]:
def round(b_1, b_2, eps):
    # returns the result for draw b_1 vs. b_2
    return np.random.binomial(1, eps[b_1, b_2]+0.5)

In [41]:
res = np.empty((1000, 1))
for i in range(1000):
    res[i] = round(1, 1, Eps)

print(sum(res))

[512.]


In [42]:
import pdb
def IF1(T, K, Eps):
    # runs interleaved filter 1 over K dueling-bandits with time-horizon T
    # win probability matrix Eps
    wins = np.zeros((K, K))
    P_hat = np.ones((K, K))*1/2
    n_rounds = np.zeros((K, K))
    c_hat = np.empty((K, K))
    delta = 1/(T*K^2)
    T_hat = 0 # total comparisons made

    def update(b_1, b_2, res):
        n_rounds[b_1, b_2] = n_rounds[b_1, b_2] + 1
        n_rounds[b_2, b_1] = n_rounds[b_2, b_1] + 1
        wins[b_1, b_2] += res
        wins[b_2, b_1] += (1-res)
        P_hat[b_1, b_2] = wins[b_1, b_2]/n_rounds[b_1, b_2]
        P_hat[b_2, b_1] = wins[b_2, b_1]/n_rounds[b_2, b_1]
        c_hat[b_1, b_2] = math.sqrt(math.log(1/delta)/n_rounds[b_1, b_2])
        c_hat[b_2, b_1] = math.sqrt(math.log(1/delta)/n_rounds[b_2, b_1])

    W = np.arange(0, K)
    b_hat = np.random.choice(K)
    W = np.array(W[~np.isin(W, b_hat)])
    while len(W) > 0:
        print(W)
        for b in W:
            res = round(b_hat, b, Eps)
            T_hat += 1
            update(b_hat, b, res)
        b_to_remove = np.where(np.logical_and(P_hat[b_hat, :] > 1/2, 
            (P_hat[b_hat, :] - c_hat[b_hat, :]) > 1/2))
        W = np.array(W[~np.isin(W, b_to_remove)])
        b_win = np.where(np.logical_and(P_hat[b_hat, :] < 1/2, 
            (P_hat[b_hat, :] + c_hat[b_hat, :]) < 1/2))[0]
        if len(b_win) > 0:
            b_hat = b_win[np.random.choice(len(b_win))]
            W = np.array(W[(~(np.isin(W, b_hat)))])
            wins = np.zeros((K, K))
            P_hat = np.ones((K, K))*1/2
            n_rounds = np.zeros((K, K))
            c_hat = np.empty((K, K))
    
    return (b_hat, T_hat)



In [43]:
print(IF1(T, K, Eps))

[0 2 3 4]
[0 2 3 4]
[0 2 3 4]
[0 2 3 4]
[0 2 3 4]
[0 2 3 4]
[0 2 3 4]
[0 2 4]
[0 2 4]
[0 2 4]
[0 2 4]
[0 2 4]
[0 2 4]
[0 2 4]
[0 2 4]
[0 2 4]
[0 2 4]
[0 2 4]
[0 2 4]
[0 2 4]
[0 2 4]
[0 2 4]
[0 2 4]
[0 2 4]
[0 2 4]
[0 2 4]
[0 2 4]
[0 2]
[0 2]
[0 2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
(4, 120)
